<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       DTW function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>DTW</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>DTW(<b>D</b>ynamics <b>T</b>ime <b>W</b>arping) measures the similarity of two temporal sequences that vary in speed or timing. It is useful when comparing two time series that have similar shapes but are shifted, scaled differently, or have different lengths. The function uses the FastDTW algorithm.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>DTW aligns the sequences by warping the time axis of one sequence to match the other, allowing for point-to-point comparisons at corresponding times. The function computes a matrix that quantifies the similarity between each point in one sequence and each point in the other sequence. It then finds the optimal path in the matrix that minimizes the total cost of warping the time axis.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>DTW is used in speech recognition, signal processing, and bioinformatics. For example, it can compare the similarity of two spoken words or to match a pattern in a noisy signal. It is also used in clustering or classification problems, where the temporal structure of the data needs to be taken into account.</p>



<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDSeries,
    DTW,
    Figure,
    plot,
    db_drop_view,
    remove_context,
    copy_to_sql,
    db_drop_table
    )

from teradatasqlalchemy.types import *

from teradataml import to_numeric
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_DTW.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we will get the time series data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("uaf", ["dtw_t1", "dtw_t2"])

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
data1 = DataFrame.from_table("dtw_t1")
data2 = DataFrame.from_table("dtw_t2")
data1 

In [ ]:
data2

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. DTW</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The DTW() function measures the similarity of two time series. The Dynamics Time Warping (DTW) algorithm is used for space and time. The underlying algorithm used by DTW() function is the FastDTW algorithm. It is not recommended for large datasets. This algorithm can find the optimal, or a close to optimal warp path between two series, depending on the search radius used.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The first step is to convert the data into TDSeries, which is required for the input time series which are passed to the DTW function.</p>


In [ ]:
data_series_df = TDSeries(data=data1,
                              id="id",
                              row_index="seq",
                              row_index_style= "TIMECODE",
                              payload_field="v",
                              payload_content="REAL")
 
data2_series_df = TDSeries(data=data2,
                          id="id",
                          row_index="seq",
                          row_index_style= "TIMECODE",
                          payload_field="v",
                          payload_content="REAL")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will execute DTW() function to measure the similarity between two time series.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Input parameters for the DTW function:</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>data1:</code> Specifies the first series input out of two.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>data2:</code> Specifies the second series input out of two.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>input_fmt_input_mode:</code> Specifies the input mode supported by the function. Permitted Values: <b>MANY2ONE</b>, <b>ONE2ONE</b>, <b>MATCH</b></li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>warp_path:</code> Specifies the type of warp paths. Permitted Values: 0,1,2,3</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>data2_filter_expr:</code> Specifies the filter expression for "data2".</li>

In [ ]:
uaf_out = DTW(data1=data_series_df,
                  data2=data2_series_df,
                  input_fmt_input_mode='MANY2ONE',
                  warp_path=2,
                  radius=1,
                  data2_filter_expr=data2_series_df.id==1)

dtw_df = uaf_out.result
dtw_df

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables created above.</p>

In [ ]:
db_drop_table("dtw_t1")
db_drop_table("dtw_t2")

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>DTW function reference: <a href = 'https://docs.teradata.com/search/all?query=DTW&content-lang=en-US'>here</a></li>
    
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>